In [1409]:
#standard ds imports
import pandas as pd
import numpy as np
#viz and stats
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
# .py imports
#import wranglerer as wr
#import modeling as md
import os
#sklearn imports
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV
import sklearn.preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score,confusion_matrix, plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
#CATboost imports
from catboost import CatBoostClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
import explore_r as ex
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB

from sklearn.feature_selection import SelectKBest, f_classif

from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('prepped_data.csv')

In [11]:
df.columns

Index(['day_of_week', 'start_time', 'week_num', 'stadium', 'temp', 'humidity',
       'wind', 'spread', 'ou', 'is_under', 'abnormal_start', 'is_playoff',
       'playoff_implications', 'is_turf', 'is_outdoor'],
      dtype='object')

In [6]:
df = df.drop(columns=['date','home_score',
       'home_wins', 'away_score', 'away_wins','total_scores'])
df['spread'] = abs(df['spread'])

In [7]:
X_train, y_train, X_validate, y_validate, X_test, y_test = ex.train_validate_test(df,'is_under')

In [8]:
X_train.shape, y_train.shape, X_validate.shape, y_validate.shape, X_test.shape, y_test.shape 

((6471, 14), (6471,), (2394, 14), (2394,), (1946, 14), (1946,))

In [ ]:
X_train.head()

# MODELING
## CATBoost

In [9]:
# Create and fit the thing
CATb = CatBoostClassifier(verbose=False,depth=5)
CATb.fit(X_train,y_train,cat_features=['day_of_week','start_time','stadium'])
CATb_preds = CATb.predict(X_train)
pd.crosstab(CATb_preds,y_train) # a confusion matrix with ACTUALS as columns and PREDICTIONS as rows

is_under,0,1
row_0,,
0,2111,1137
1,1069,2154


In [10]:
print(f'Accuracy-Train {round(CATb.score(X_train,y_train),4)}')
print(f'Accuracy-Validate {round(CATb.score(X_validate,y_validate),4)}')
print(classification_report(y_train,CATb_preds))
print(classification_report(y_validate,CATb.predict(X_validate)))

Accuracy-Train 0.6591
Accuracy-Validate 0.5063
              precision    recall  f1-score   support

           0       0.65      0.66      0.66      3180
           1       0.67      0.65      0.66      3291

    accuracy                           0.66      6471
   macro avg       0.66      0.66      0.66      6471
weighted avg       0.66      0.66      0.66      6471

              precision    recall  f1-score   support

           0       0.50      0.51      0.50      1178
           1       0.51      0.51      0.51      1216

    accuracy                           0.51      2394
   macro avg       0.51      0.51      0.51      2394
weighted avg       0.51      0.51      0.51      2394



## CATboost grid_search CROSS_VALIDATION k=5

In [14]:
param_grid = {
    'verbose': [False],
    'depth': [5, 10,15]
}
gr_search = GridSearchCV(CatBoostClassifier(),
                      param_grid)

In [15]:
gr_search

GridSearchCV(estimator=<catboost.core.CatBoostClassifier object at 0x12f8fd2e0>,
             param_grid={'depth': [5, 10, 15], 'verbose': [False]})

In [16]:
gr_search.fit(X_train, y_train,cat_features=['day_of_week','start_time','stadium'])

GridSearchCV(estimator=<catboost.core.CatBoostClassifier object at 0x12f8fd2e0>,
             param_grid={'depth': [5, 10, 15], 'verbose': [False]})

In [17]:
results = gr_search.cv_results_
results_df_init = pd.DataFrame(results)
results_df_init.shape

(3, 15)

In [18]:
params = pd.DataFrame(results['params'])
results_df_init.sort_values(by='rank_test_score').head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_depth,param_verbose,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,2.767949,0.020285,0.002929,0.000220,5,False,"{'depth': 5, 'verbose': False}",0.528185,0.512365,0.513910,0.515456,0.525502,0.519084,0.006467,1
1,6.124596,0.038884,0.004495,0.000132,10,False,"{'depth': 10, 'verbose': False}",0.498842,0.503864,0.518547,0.507728,0.513138,0.508424,0.006896,2
2,87.300167,1.279427,0.039538,0.034079,15,False,"{'depth': 15, 'verbose': False}",0.495753,0.513138,0.505410,0.512365,0.510046,0.507342,0.006390,3


# start removing features 
### Decision Tree

In [227]:
df = pd.read_csv('prepped_data.csv')

In [228]:
df.columns

Index(['date', 'day_of_week', 'start_time', 'week_num', 'home_score',
       'home_wins', 'away_score', 'away_wins', 'stadium', 'temp', 'humidity',
       'wind', 'spread', 'ou', 'is_under', 'abnormal_start', 'total_scores',
       'is_playoff', 'playoff_implications', 'is_turf', 'is_outdoor'],
      dtype='object')

In [229]:
df.drop(columns=['date', 'day_of_week', 'spread', 'home_score',
       'home_wins', 'away_score', 'away_wins', 'stadium',
        'abnormal_start', 'total_scores', 'is_outdoor'], inplace=True)

In [230]:
df

,start_time,week_num,temp,humidity,wind,ou,is_under,is_playoff,playoff_implications,is_turf
0,6,19,72,0,0,51.0,0,1,1,0
1,3,19,52,48,14,45.5,1,1,1,0
2,6,19,22,55,13,48.0,1,1,1,0
3,3,19,32,10,0,49.0,1,1,1,1
4,6,19,55,47,19,47.0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...
10806,4,1,76,71,8,37.0,1,0,0,1
10807,4,1,73,76,10,41.0,0,0,0,0
10808,4,1,70,77,10,36.5,0,0,0,0
10809,4,1,72,0,0,42.5,0,0,0,1


In [231]:
X_train, y_train, X_validate, y_validate, X_test, y_test = ex.train_validate_test(df,'is_under')

In [259]:
depth = [i for i in range(1, 21, 1)] + [None]
param_grid = {
    'criterion':['gini', "entropy", "log_loss"],
    'splitter':['best'],
    'max_depth': depth,
    'min_samples_split':[i for i in range(1, 21, 1)],
    'min_samples_leaf':[i for i in range(1, 21, 1)],
}
gr_search = GridSearchCV(DecisionTreeClassifier(),
                      param_grid)

In [260]:
gr_search

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, 16, 17, 18, 19, 20, None],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                              12, 13, 14, 15, 16, 17, 18, 19,
                                              20],
                         'min_samples_split': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                                               11, 12, 13, 14, 15, 16, 17, 18,
                                               19, 20],
                         'splitter': ['best']})

In [261]:
gr_search.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, 16, 17, 18, 19, 20, None],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                              12, 13, 14, 15, 16, 17, 18, 19,
                                              20],
                         'min_samples_split': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                                               11, 12, 13, 14, 15, 16, 17, 18,
                                               19, 20],
                         'splitter': ['best']})

In [262]:
results = gr_search.cv_results_
results_df_init = pd.DataFrame(results)
results_df_init.shape

(25200, 18)

In [263]:
results_df_init.loc[results_df_init['mean_test_score'].idxmax()]

mean_fit_time                                                       0.005544
std_fit_time                                                        0.000097
mean_score_time                                                       0.0005
std_score_time                                                      0.000006
param_criterion                                                      entropy
param_max_depth                                                            6
param_min_samples_leaf                                                     5
param_min_samples_split                                                    2
param_splitter                                                          best
params                     {'criterion': 'entropy', 'max_depth': 6, 'min_...
split0_test_score                                                   0.505792
split1_test_score                                                   0.528594
split2_test_score                                                   0.523184

In [264]:
results_df_init[results_df_init['mean_test_score'] == results_df_init['mean_test_score'].max()]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
10481,0.005544,0.000097,0.000500,0.000006,entropy,6,5,2,best,"{'criterion': 'entropy', 'max_depth': 6, 'min_...",0.505792,0.528594,0.523184,0.528594,0.523184,0.521869,0.008395,1
10482,0.005527,0.000034,0.000503,0.000006,entropy,6,5,3,best,"{'criterion': 'entropy', 'max_depth': 6, 'min_...",0.505792,0.528594,0.523184,0.528594,0.523184,0.521869,0.008395,1
10485,0.005484,0.000039,0.000550,0.000094,entropy,6,5,6,best,"{'criterion': 'entropy', 'max_depth': 6, 'min_...",0.505792,0.528594,0.523184,0.528594,0.523184,0.521869,0.008395,1
10487,0.005513,0.000048,0.000503,0.000007,entropy,6,5,8,best,"{'criterion': 'entropy', 'max_depth': 6, 'min_...",0.505792,0.528594,0.523184,0.528594,0.523184,0.521869,0.008395,1
10489,0.005523,0.000062,0.000506,0.000015,entropy,6,5,10,best,"{'criterion': 'entropy', 'max_depth': 6, 'min_...",0.505792,0.528594,0.523184,0.528594,0.523184,0.521869,0.008395,1
10490,0.005552,0.000075,0.000535,0.000043,entropy,6,5,11,best,"{'criterion': 'entropy', 'max_depth': 6, 'min_...",0.505792,0.528594,0.523184,0.528594,0.523184,0.521869,0.008395,1
10491,0.005484,0.000029,0.000503,0.000003,entropy,6,5,12,best,"{'criterion': 'entropy', 'max_depth': 6, 'min_...",0.505792,0.528594,0.523184,0.528594,0.523184,0.521869,0.008395,1
18882,0.005751,0.000134,0.000619,0.000089,log_loss,6,5,3,best,"{'criterion': 'log_loss', 'max_depth': 6, 'min...",0.505792,0.528594,0.523184,0.528594,0.523184,0.521869,0.008395,1
18883,0.005602,0.000119,0.000550,0.000035,log_loss,6,5,4,best,"{'criterion': 'log_loss', 'max_depth': 6, 'min...",0.505792,0.528594,0.523184,0.528594,0.523184,0.521869,0.008395,1
18884,0.005513,0.000059,0.000508,0.000015,log_loss,6,5,5,best,"{'criterion': 'log_loss', 'max_depth': 6, 'min...",0.505792,0.528594,0.523184,0.528594,0.523184,0.521869,0.008395,1


In [265]:
param_grid = {
    'criterion': ['gini'],
    'max_depth': [6],
      'min_samples_leaf': [5],
      'min_samples_split': [2],
      'splitter': ['best']}
gr_search1 = GridSearchCV(DecisionTreeClassifier(),
                      param_grid)

In [266]:
gr_search1


GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini'], 'max_depth': [6],
                         'min_samples_leaf': [5], 'min_samples_split': [2],
                         'splitter': ['best']})

In [267]:
gr_search1.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini'], 'max_depth': [6],
                         'min_samples_leaf': [5], 'min_samples_split': [2],
                         'splitter': ['best']})

In [268]:
results1 = gr_search1.cv_results_
results_df_init1 = pd.DataFrame(results1)
results_df_init1.shape

(1, 18)

In [269]:
results_df_init1

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.010926,0.00622,0.001102,0.000564,gini,6,5,2,best,"{'criterion': 'gini', 'max_depth': 6, 'min_sam...",0.495753,0.507728,0.508501,0.517774,0.527048,0.511361,0.010509,1


# Trying something different

In [1479]:
 def train_validate_test(df,target):
    """
    Splits data into 3 segments and stratifies on target
    requires the dataframe and target as args
    """
    train_val, test = train_test_split(df,
                                       train_size=0.8,
                                       random_state=706,
                                       stratify=df[target])
    train, validate = train_test_split(train_val,
                                       train_size=0.7,
                                       random_state=706,
                                       stratify=train_val[target])
    return train, validate, test

In [1883]:
df = pd.read_csv('prepped_data.csv')

In [1884]:
# df = pd.get_dummies(df, columns=['day_of_week', 'stadium'])

In [1885]:
columns = df.columns.to_list()

In [1886]:
columns.remove('is_under')
columns.remove('date')
columns.remove('total_scores')
columns.remove('home_score')
columns.remove('away_score')
columns.remove('day_of_week')
columns.remove('stadium')

In [2471]:
def scale_data(train, val, test):
    x_cols = columns
    split = [train, val, test]
    scale_list= []
    scaler = MinMaxScaler()
    scaler.fit(train[x_cols])
    for cut in split:
        cut_copy = cut.copy()
        cut_copy[x_cols] = scaler.transform(cut_copy[x_cols])
        scale_list.append(cut_copy)
    return scale_list[0], scale_list[1], scale_list[2] 

In [1888]:
train, val, test = train_validate_test(df, 'is_under')

In [1889]:
train.shape, val.shape, test.shape

((6053, 21), (2595, 21), (2163, 21))

In [1890]:
train


,date,day_of_week,start_time,week_num,home_score,home_wins,away_score,away_wins,stadium,temp,...,wind,spread,ou,is_under,abnormal_start,total_scores,is_playoff,playoff_implications,is_turf,is_outdoor
10538,1980-09-28,Sunday,1,4,27,3,7,2,Pontiac Silverdome,72,...,0,6.0,43.0,1,0,34,0,0,1,0
1913,2016-01-03,Sunday,4,17,23,10,17,7,Arrowhead Stadium,37,...,9,7.0,43.0,1,0,40,0,1,0,1
9647,1984-10-28,Sunday,1,9,13,0,31,2,Houston Astrodome,72,...,0,4.5,40.0,0,0,44,0,0,1,0
9828,1983-11-24,Thursday,4,13,35,10,17,5,Texas Stadium,43,...,8,10.0,54.5,1,1,52,0,1,1,1
8253,1990-12-23,Sunday,1,16,27,10,14,6,Soldier Field,8,...,12,6.5,35.0,0,0,41,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5080,2003-12-14,Sunday,1,15,28,9,26,6,The Coliseum,37,...,10,5.0,37.0,0,0,54,0,1,0,1
4856,2004-11-28,Sunday,1,12,27,6,16,6,Hubert H. Humphrey Metrodome,72,...,0,6.0,45.5,1,0,43,0,1,1,0
6428,1998-11-15,Sunday,4,11,20,6,17,5,Network Associates Coliseum,58,...,10,1.5,38.0,1,0,37,0,1,0,1
9,2023-01-15,Sunday,4,19,24,13,31,9,U.S. Bank Stadium,72,...,0,3.0,48.0,0,0,55,1,1,1,0


In [1891]:
train_scaled, val_scaled, test_scaled = scale_data(train, val, test)



In [1892]:
train_scaled

,date,day_of_week,start_time,week_num,home_score,home_wins,away_score,away_wins,stadium,temp,...,wind,spread,ou,is_under,abnormal_start,total_scores,is_playoff,playoff_implications,is_turf,is_outdoor
10538,1980-09-28,Sunday,-0.779424,-1.075566,27,-0.330083,7,-0.652270,Pontiac Silverdome,0.745785,...,-1.124650,0.178307,0.181015,1,-0.483264,34,-0.213127,-0.890776,1.074287,-1.789010
1913,2016-01-03,Sunday,0.276497,1.415096,23,1.724859,17,0.856401,Arrowhead Stadium,-1.451870,...,0.345089,0.468713,0.181015,1,-0.483264,40,-0.213127,1.122617,-0.930850,0.558969
9647,1984-10-28,Sunday,-0.779424,-0.117619,13,-1.210772,31,-0.652270,Houston Astrodome,0.745785,...,-1.124650,-0.257302,-0.448592,0,-0.483264,44,-0.213127,-0.890776,1.074287,-1.789010
9828,1983-11-24,Thursday,0.276497,0.648739,35,1.724859,17,0.252932,Texas Stadium,-1.075129,...,0.181785,1.339930,2.594506,1,2.069263,52,-0.213127,1.122617,1.074287,0.558969
8253,1990-12-23,Sunday,-0.779424,1.223507,27,1.724859,14,0.554667,Soldier Field,-3.272784,...,0.835002,0.323510,-1.497936,0,-0.483264,41,-0.213127,1.122617,-0.930850,0.558969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5080,2003-12-14,Sunday,-0.779424,1.031918,28,1.431296,26,0.554667,The Coliseum,-1.451870,...,0.508394,-0.112099,-1.078198,0,-0.483264,54,-0.213127,1.122617,-0.930850,0.558969
4856,2004-11-28,Sunday,-0.779424,0.457149,27,0.550606,16,0.554667,Hubert H. Humphrey Metrodome,0.745785,...,-1.124650,0.178307,0.705687,1,-0.483264,43,-0.213127,1.122617,1.074287,-1.789010
6428,1998-11-15,Sunday,0.276497,0.265560,20,0.550606,17,0.252932,Network Associates Coliseum,-0.133277,...,0.508394,-1.128519,-0.868329,1,-0.483264,37,-0.213127,1.122617,-0.930850,0.558969
9,2023-01-15,Sunday,0.276497,1.798275,24,2.605548,31,1.459869,U.S. Bank Stadium,0.745785,...,-1.124650,-0.692910,1.230359,0,-0.483264,55,4.692037,1.122617,1.074287,-1.789010


In [1893]:
baseline_accuracy = train.is_under.value_counts(normalize=True)[1]

In [1894]:
baseline_accuracy

0.5073517264166529

In [1895]:
def get_target_and_features(train_scale, val_scale, test_scale):    
    x_cols = columns
    y_cols = 'is_under'

    x_train = train_scale[x_cols]
    y_train = train_scale[y_cols]

    x_val = val_scale[x_cols]
    y_val = val_scale[y_cols]

    x_test = test_scale[x_cols]
    y_test = test_scale[y_cols]
    return x_train, y_train, x_val, y_val, x_test, y_test

In [1896]:
x_train, y_train, x_val, y_val, x_test, y_test = get_target_and_features(train_scaled, val_scaled, test_scaled)

In [1860]:
def get_decisionTree_model(depth):
    """
    Returns a decision treen model with a max depth arg
    prints out the Accuracy of train and validate and the 
    classification report
    """
    clf = DecisionTreeClassifier(max_depth=depth, random_state=706)
    #class_weight='balanced'
    # fit the thing
    clf.fit(x_train, y_train)

    model_proba = clf.predict_proba(x_train)
    model_preds = clf.predict(x_train)

    model_score = clf.score(x_train, y_train)

    #classification report:
    print(
        classification_report(y_train,
                          model_preds))
    print('Accuracy of Random Tree classifier on training set: {:.3f}'
     .format(clf.score(x_train, y_train)))
    print('Accuracy of Random Tree classifier on validation set: {:.3f}'
     .format(clf.score(x_val, y_val)))
    return clf, model_preds

In [2031]:
get_decisionTree_model(10)

              precision    recall  f1-score   support

           0       0.61      0.68      0.65      2093
           1       0.65      0.58      0.61      2139

    accuracy                           0.63      4232
   macro avg       0.63      0.63      0.63      4232
weighted avg       0.63      0.63      0.63      4232

Accuracy of Random Tree classifier on training set: 0.629
Accuracy of Random Tree classifier on validation set: 0.497


(DecisionTreeClassifier(max_depth=10, random_state=706),
 array([1, 0, 1, ..., 0, 1, 1]))

In [790]:
x_train

,start_time,week_num,home_wins,away_wins,temp,humidity,wind,spread,ou,abnormal_start,...,stadium_The Coliseum,stadium_Three Rivers Stadium,stadium_Tiger Stadium (LSU),stadium_Tottenham Stadium,stadium_Twickenham Stadium,stadium_U.S. Bank Stadium,stadium_University of Phoenix Stadium,stadium_Vanderbilt Stadium,stadium_Veterans Stadium,stadium_Wembley Stadium
10538,0.000000,0.166667,0.15,0.10,0.742268,0.000000,0.000000,0.226415,0.422535,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1913,0.272727,0.888889,0.50,0.35,0.381443,0.565657,0.128571,0.264151,0.422535,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9647,0.000000,0.444444,0.00,0.10,0.742268,0.000000,0.000000,0.169811,0.338028,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9828,0.272727,0.666667,0.50,0.25,0.443299,0.616162,0.114286,0.377358,0.746479,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8253,0.000000,0.833333,0.50,0.30,0.082474,0.686869,0.171429,0.245283,0.197183,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5080,0.000000,0.777778,0.45,0.30,0.381443,0.898990,0.142857,0.188679,0.253521,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4856,0.000000,0.611111,0.30,0.30,0.742268,0.000000,0.000000,0.226415,0.492958,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6428,0.272727,0.555556,0.30,0.25,0.597938,0.656566,0.142857,0.056604,0.281690,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.272727,1.000000,0.65,0.45,0.742268,0.000000,0.000000,0.113208,0.563380,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [560]:
def get_random_forest():
    """
    Runs through two for loops from range 1 - 5 each time increasing the max depth 
    and min sample leaf
    puts all of the models in a pandas data frame and sorts for the hightes valadation 
    Prints out the classification report on the best model
    """
    
    model_list = []

    for j in range (1, 15):
        for i in range(2, 15):
            rf = RandomForestClassifier(n_estimators=101 ,max_depth=i, min_samples_leaf=j, random_state=706)

            rf = rf.fit(x_train, y_train)
            train_accuracy = rf.score(x_train, y_train)
            validate_accuracy = rf.score(x_val, y_val)
            model_preds = rf.predict(x_train)

            output = {
                "min_samples_per_leaf": j,
                "max_depth": i,
                "train_accuracy": train_accuracy,
                "validate_accuracy": validate_accuracy,
                'model_preds': model_preds
            }
            model_list.append(output)
            
    df = pd.DataFrame(model_list)
    df["difference"] = df.train_accuracy - df.validate_accuracy
    df["baseline_accuracy"] = baseline_accuracy
    # df[df.validate_accuracy > df.baseline_accuracy + .05].sort_values(by=['difference'], ascending=True).head(15)
    df.sort_values(by=['validate_accuracy'], ascending=False).head(1)
    
    #classification report:
    print(classification_report(y_train, df['model_preds'][1]))
    return df.sort_values(by=['validate_accuracy'], ascending=False).head(1)
    


In [2026]:
get_random_forest()

              precision    recall  f1-score   support

           0       0.54      0.56      0.55      2093
           1       0.56      0.53      0.54      2139

    accuracy                           0.55      4232
   macro avg       0.55      0.55      0.55      4232
weighted avg       0.55      0.55      0.55      4232



,min_samples_per_leaf,max_depth,train_accuracy,validate_accuracy,model_preds,difference,baseline_accuracy
154,12,13,0.670132,0.520948,"[1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, ...",0.149184,0.507352


In [2559]:
def get_logReg_model(data):
    """
    build a logistical regression model and prints out the accuracy on training and validation along with the classification report. 
    Must type in train_val as your data arrg to get the train val result.
    Type test if you want to test the model
    if you want a csv of the model preds and preds proba then un comment all of the stuff at the bottom
    """
    logit = LogisticRegression(random_state=706)
    logit.fit(x_train, y_train)
    y_pred = logit.predict(x_train)
    y_pred_val = logit.predict(x_val)
    y_pred_test = logit.predict(x_test)
    y_proba = logit.predict_proba(x_train)
    logit_val = logit.predict(x_val)
    
    if data == 'train_val':
        print('Accuracy of Logistic Regression classifier on training set: {:.3f}'
         .format(logit.score(x_train, y_train)))
        print('Accuracy of Logistic Regression classifier on validation set: {:.3f}'
         .format(logit.score(x_val, y_val)))
        print(
        classification_report(y_train,
                          y_pred))
        print(
        classification_report(y_val,
                          y_pred_val))
    else: 
        print('Accuracy of logistic regression classifier on test set: {:.3f}'
         .format(logit.score(x_test, y_test)))
        print(
        classification_report(y_test,
                          y_pred_test))

In [2025]:
get_logReg_model('train_val')


Accuracy of Logistic Regression classifier on training set: 0.513
Accuracy of Logistic Regression classifier on validation set: 0.517
              precision    recall  f1-score   support

           0       0.51      0.53      0.52      2093
           1       0.52      0.50      0.51      2139

    accuracy                           0.51      4232
   macro avg       0.51      0.51      0.51      4232
weighted avg       0.51      0.51      0.51      4232

              precision    recall  f1-score   support

           0       0.51      0.53      0.52       897
           1       0.52      0.50      0.51       917

    accuracy                           0.52      1814
   macro avg       0.52      0.52      0.52      1814
weighted avg       0.52      0.52      0.52      1814



In [2556]:
bnb = BernoulliNB()
# Fit the model using the training data
bnb.fit(x_train, y_train)

# Make predictions on the testing data
y_pred = bnb.predict(x_train)
y_pred_val = bnb.predict(x_val)

# Calculate the accuracy of the model
print(classification_report(y_train,
                          y_pred))
print('Accuracy of Logistic Regression classifier on training set: {:.3f}'
 .format(bnb.score(x_train, y_train)))
print('Accuracy of Logistic Regression classifier on validation set: {:.3f}'
 .format(bnb.score(x_val, y_val)))
print(classification_report(y_val,
                          y_pred_val))

              precision    recall  f1-score   support

           0       0.51      0.39      0.44      2093
           1       0.51      0.63      0.57      2139

    accuracy                           0.51      4232
   macro avg       0.51      0.51      0.50      4232
weighted avg       0.51      0.51      0.50      4232

Accuracy of Logistic Regression classifier on training set: 0.511
Accuracy of Logistic Regression classifier on validation set: 0.523
              precision    recall  f1-score   support

           0       0.52      0.39      0.45       897
           1       0.52      0.65      0.58       917

    accuracy                           0.52      1814
   macro avg       0.52      0.52      0.51      1814
weighted avg       0.52      0.52      0.52      1814



In [2494]:
k = 8

x_train.columns

x = train[['start_time', 'week_num', 'home_wins', 'away_wins', 'temp', 'humidity',
       'wind', 'spread', 'ou', 'abnormal_start', 'is_playoff',
       'playoff_implications', 'is_turf', 'is_outdoor']]

y = y_train

y_train


selector = SelectKBest(score_func=f_classif, k=k)

selector.fit(x, y)

mask = selector.get_support()

X_selected = x.iloc[:, mask]


In [2495]:
X_selected.columns.to_list()

['home_wins',
 'away_wins',
 'humidity',
 'wind',
 'abnormal_start',
 'is_playoff',
 'is_turf',
 'is_outdoor']

In [761]:
columns = X_selected.columns.to_list()

In [555]:
df[columns]

,start_time,home_wins,humidity,wind,ou,is_turf,is_outdoor
0,6,16,0,0,51.0,0,0
1,3,15,48,14,45.5,0,1
2,6,15,55,13,48.0,0,1
3,3,14,10,0,49.0,1,1
4,6,14,47,19,47.0,0,1
...,...,...,...,...,...,...,...
10806,4,0,71,8,37.0,1,1
10807,4,0,76,10,41.0,0,1
10808,4,0,77,10,36.5,0,1
10809,4,0,0,0,42.5,1,0


## Neural Net sklearn

In [565]:
from sklearn.neural_network import MLPClassifier

In [1516]:
x_train

,start_time,week_num,home_wins,away_wins,temp,humidity,wind,spread,ou,abnormal_start,is_playoff,playoff_implications,is_turf,is_outdoor
10538,0.000000,0.166667,0.15,0.10,0.742268,0.000000,0.000000,0.226415,0.422535,0.0,0.0,0.0,1.0,0.0
1913,0.272727,0.888889,0.50,0.35,0.381443,0.565657,0.128571,0.264151,0.422535,0.0,0.0,1.0,0.0,1.0
9647,0.000000,0.444444,0.00,0.10,0.742268,0.000000,0.000000,0.169811,0.338028,0.0,0.0,0.0,1.0,0.0
9828,0.272727,0.666667,0.50,0.25,0.443299,0.616162,0.114286,0.377358,0.746479,1.0,0.0,1.0,1.0,1.0
8253,0.000000,0.833333,0.50,0.30,0.082474,0.686869,0.171429,0.245283,0.197183,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5080,0.000000,0.777778,0.45,0.30,0.381443,0.898990,0.142857,0.188679,0.253521,0.0,0.0,1.0,0.0,1.0
4856,0.000000,0.611111,0.30,0.30,0.742268,0.000000,0.000000,0.226415,0.492958,0.0,0.0,1.0,1.0,0.0
6428,0.272727,0.555556,0.30,0.25,0.597938,0.656566,0.142857,0.056604,0.281690,0.0,0.0,1.0,0.0,1.0
9,0.272727,1.000000,0.65,0.45,0.742268,0.000000,0.000000,0.113208,0.563380,0.0,1.0,1.0,1.0,0.0


In [996]:
# x_train.columns.to_list()

In [2557]:
# Create an MLP classifier object
clf = MLPClassifier(hidden_layer_sizes=(120, 60, 30), activation='relu', solver='adam', learning_rate='constant', random_state=706)

# Fit the model to the training data
clf.fit(x_train, y_train)


model_proba = clf.predict_proba(x_train)
model_preds = clf.predict(x_train)
model_preds_val = clf.predict(x_val)
model_score = clf.score(x_train, y_train)

#classification report:
print(
    classification_report(y_train,
                      model_preds))
print('Accuracy of Random Tree classifier on training set: {:.3f}'
 .format(clf.score(x_train, y_train)))
print('Accuracy of Random Tree classifier on validation set: {:.3f}'
 .format(clf.score(x_val, y_val)))
print(
    classification_report(y_val,
                      model_preds_val))

              precision    recall  f1-score   support

           0       0.57      0.62      0.59      2093
           1       0.59      0.54      0.57      2139

    accuracy                           0.58      4232
   macro avg       0.58      0.58      0.58      4232
weighted avg       0.58      0.58      0.58      4232

Accuracy of Random Tree classifier on training set: 0.579
Accuracy of Random Tree classifier on validation set: 0.509
              precision    recall  f1-score   support

           0       0.50      0.53      0.52       897
           1       0.52      0.49      0.50       917

    accuracy                           0.51      1814
   macro avg       0.51      0.51      0.51      1814
weighted avg       0.51      0.51      0.51      1814



## Knn

In [2558]:
k = 3
knn = KNeighborsClassifier(n_neighbors=k)

knn.fit(x_train, y_train)

y_pred = knn.predict(x_val)

print('Accuracy of Logistic Regression classifier on training set: {:.2f}'
 .format(knn.score(x_train, y_train)))
print('Accuracy of Logistic Regression classifier on validation set: {:.2f}'
 .format(knn.score(x_val, y_val)))

print(classification_report(y_val,
                          y_pred))

Accuracy of Logistic Regression classifier on training set: 0.74
Accuracy of Logistic Regression classifier on validation set: 0.52
              precision    recall  f1-score   support

           0       0.51      0.52      0.51       897
           1       0.52      0.51      0.52       917

    accuracy                           0.52      1814
   macro avg       0.52      0.52      0.52      1814
weighted avg       0.52      0.52      0.52      1814



In [ ]:
def scale_df(df):
    x_cols = columns
    scaler = StandardScaler()
    scaler.fit(df[x_cols])
    df[x_cols] = scaler.transform(df[x_cols])
    return df
    

# DBScan

In [1774]:
x_train

,start_time,week_num,home_wins,away_wins,temp,humidity,wind,spread,ou,abnormal_start,is_playoff,playoff_implications,is_turf,is_outdoor
10538,-0.779424,-1.075566,-0.330083,-0.652270,0.745785,-1.326768,-1.124650,0.178307,0.181015,-0.483264,-0.213127,-0.890776,1.074287,-1.789010
1913,0.276497,1.415096,1.724859,0.856401,-1.451870,0.381557,0.345089,0.468713,0.181015,-0.483264,-0.213127,1.122617,-0.930850,0.558969
9647,-0.779424,-0.117619,-1.210772,-0.652270,0.745785,-1.326768,-1.124650,-0.257302,-0.448592,-0.483264,-0.213127,-0.890776,1.074287,-1.789010
9828,0.276497,0.648739,1.724859,0.252932,-1.075129,0.534086,0.181785,1.339930,2.594506,2.069263,-0.213127,1.122617,1.074287,0.558969
8253,-0.779424,1.223507,1.724859,0.554667,-3.272784,0.747626,0.835002,0.323510,-1.497936,-0.483264,-0.213127,1.122617,-0.930850,0.558969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5080,-0.779424,1.031918,1.431296,0.554667,-1.451870,1.388248,0.508394,-0.112099,-1.078198,-0.483264,-0.213127,1.122617,-0.930850,0.558969
4856,-0.779424,0.457149,0.550606,0.554667,0.745785,-1.326768,-1.124650,0.178307,0.705687,-0.483264,-0.213127,1.122617,1.074287,-1.789010
6428,0.276497,0.265560,0.550606,0.252932,-0.133277,0.656109,0.508394,-1.128519,-0.868329,-0.483264,-0.213127,1.122617,-0.930850,0.558969
9,0.276497,1.798275,2.605548,1.459869,0.745785,-1.326768,-1.124650,-0.692910,1.230359,-0.483264,4.692037,1.122617,1.074287,-1.789010


In [2561]:
df = pd.read_csv('prepped_data.csv')

# Convert 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Filter out years less than 1993
df = df[df['date'].dt.year > 1993]
# df = pd.get_dummies(df, columns=['stadium']) 

In [2562]:
df.date.min()

Timestamp('1994-01-02 00:00:00')

In [2563]:
df.columns.to_list()

['date',
 'day_of_week',
 'start_time',
 'week_num',
 'home_score',
 'home_wins',
 'away_score',
 'away_wins',
 'stadium',
 'temp',
 'humidity',
 'wind',
 'spread',
 'ou',
 'is_under',
 'abnormal_start',
 'total_scores',
 'is_playoff',
 'playoff_implications',
 'is_turf',
 'is_outdoor']

In [2564]:
# df = pd.read_csv('prepped_data.csv')
columns = df.columns.to_list()
columns.remove('is_under')
columns.remove('date')
columns.remove('total_scores')
columns.remove('home_score')
columns.remove('away_score')
columns.remove('day_of_week')
columns.remove('stadium')
train, val, test = train_validate_test(df, 'is_under')
train_scaled, val_scaled, test_scaled = scale_data(train, val, test)
x_train, y_train, x_val, y_val, x_test, y_test = get_target_and_features(train_scaled, val_scaled, test_scaled)

In [2565]:
cols = [ 'wind', 'temp', 'ou','is_turf','away_wins']

x_train = x_train[cols]
x_val = x_val[cols]
x_test= x_test[cols]

In [2566]:
get_logReg_model('train_val')

Accuracy of Logistic Regression classifier on training set: 0.520
Accuracy of Logistic Regression classifier on validation set: 0.540
              precision    recall  f1-score   support

           0       0.52      0.46      0.49      2093
           1       0.52      0.58      0.55      2139

    accuracy                           0.52      4232
   macro avg       0.52      0.52      0.52      4232
weighted avg       0.52      0.52      0.52      4232

              precision    recall  f1-score   support

           0       0.54      0.47      0.50       897
           1       0.54      0.61      0.57       917

    accuracy                           0.54      1814
   macro avg       0.54      0.54      0.54      1814
weighted avg       0.54      0.54      0.54      1814



In [2567]:
get_logReg_model('test')

Accuracy of logistic regression classifier on test set: 0.511
              precision    recall  f1-score   support

           0       0.51      0.47      0.49       748
           1       0.52      0.55      0.53       764

    accuracy                           0.51      1512
   macro avg       0.51      0.51      0.51      1512
weighted avg       0.51      0.51      0.51      1512



In [2313]:
# from sklearn.cluster import DBSCAN
# # Define the columns to drop
# drop_cols = ['start_time']

# # Create an instance of DBSCAN
# dbscan = DBSCAN(eps=0.009, min_samples=15)
# dbscan.fit(x_train.drop(columns=['start_time','humidity', 'wind', 'temp',]))
# labels = dbscan.labels_

# # Access the core samples identified by DBSCAN
# core_samples = dbscan.core_sample_indices_

# # Print the number of clusters (excluding noise points)
# num_clusters = len(set(labels)) - (1 if -1 in labels else 0)
# print("Number of clusters:", num_clusters)

# # Print the cluster labels and core sample indices
# print("Cluster labels:", labels)
# print("Core samples indices:", core_samples)

# x_train['cluster']= labels

# # Create dummy variables for the 'cluster' column in x_train
# x_train = pd.get_dummies(x_train, columns=['cluster'])

# # Add missing cluster columns to x_val and fill with zeros
# labels_val = dbscan.fit_predict(x_val)

# # Add cluster labels to x_val
# x_val['cluster'] = labels_val

# # Create dummy variables for the 'cluster' column in x_val
# x_val = pd.get_dummies(x_val, columns=['cluster'])

# for col in x_train.columns.to_list():
#     if col not in x_val.columns.to_list():
#         x_val[col] = 0


In [2475]:
x_val

,wind,temp,ou,is_turf,away_wins
3842,0.000000,0.742268,0.522388,0.0,0.10
7183,0.000000,0.742268,0.223881,1.0,0.15
2376,0.100000,0.721649,0.552239,1.0,0.10
7185,0.100000,0.628866,0.388060,0.0,0.25
3718,0.000000,0.742268,0.447761,0.0,0.00
...,...,...,...,...,...
7415,0.000000,0.742268,0.253731,1.0,0.15
1320,0.000000,0.824742,0.567164,0.0,0.05
4801,0.042857,0.474227,0.417910,1.0,0.25
3436,0.042857,0.896907,0.208955,0.0,0.05


In [1901]:
x_val

,start_time,humidity,wind,temp,cluster_-1,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,...,cluster_7,cluster_8,cluster_9,cluster_10,cluster_11,cluster_12,cluster_13,cluster_14,cluster_15,cluster_16
749,0.276497,0.137510,0.835002,1.750427,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8954,0.276497,-0.167547,1.651524,-0.447228,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1462,0.276497,-1.326768,-1.124650,0.745785,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6819,0.276497,0.107005,0.998307,-1.514660,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7223,0.276497,1.205213,0.345089,0.431834,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5692,-0.779424,-1.326768,-0.144824,0.808575,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1640,-0.779424,-1.326768,-1.124650,-0.510018,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10168,3.092286,0.717120,-0.144824,-1.765821,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9954,-0.779424,0.351051,1.324916,1.499266,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
